In [19]:
# Importing dependencies
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [20]:
# Read data file and inspect dataset
file_path = os.path.join('data', 'Amazon Sale Report.csv')
amazon_sales = pd.read_csv(file_path)

print(amazon_sales.columns, "\n")
display(amazon_sales.head())

Index(['index', 'Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel ',
       'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN',
       'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city',
       'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids',
       'B2B', 'fulfilled-by', 'Unnamed: 22'],
      dtype='object') 



C:\Users\husse\AppData\Local\Temp\ipykernel_4708\3517283731.py:3: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  amazon_sales = pd.read_csv(file_path)


,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN


# Data Munging

We'll start by normalizing column names, dtypes, and values

In [ ]:
# Normalize column names and inspect info
amazon_sales.columns = (amazon_sales.columns.str.strip()
                        .str.replace(r'[-\s]', '_', regex=True)
                        .str.lower())
amazon_sales.rename(columns={'qty': 'quantity', 'fulfilment': 'fulfillment'}, inplace=True)
amazon_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               128975 non-null  int64  
 1   order_id            128975 non-null  object 
 2   date                128975 non-null  object 
 3   status              128975 non-null  object 
 4   fulfillment         128975 non-null  object 
 5   sales_channel       128975 non-null  object 
 6   ship_service_level  128975 non-null  object 
 7   style               128975 non-null  object 
 8   sku                 128975 non-null  object 
 9   category            128975 non-null  object 
 10  size                128975 non-null  object 
 11  asin                128975 non-null  object 
 12  courier_status      122103 non-null  object 
 13  quantity            128975 non-null  int64  
 14  currency            121180 non-null  object 
 15  amount              121180 non-nul

In [ ]:
# Drop unnamed column and convert date types
amazon_sales.drop(columns='unnamed:_22', inplace=True)
amazon_sales['date'] = pd.to_datetime(
    amazon_sales['date'], 
    format='%m-%d-%y'
)

categorical_columns = [
    'status', 'fulfillment', 'sales_channel', 'ship_service_level', 
    'category', 'size', 'courier_status', 'currency', 'ship_state',
    'ship_country', 'fulfilled_by'
]

string_columns = [
    'order_id', 'style', 'sku', 'asin', 'ship_city', 'promotion_ids',
    'ship_postal_code'
]


amazon_sales[categorical_columns] = amazon_sales[categorical_columns].astype('category')
amazon_sales[string_columns] = amazon_sales[string_columns].astype('string')

amazon_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   index               128975 non-null  int64         
 1   order_id            128975 non-null  string        
 2   date                128975 non-null  datetime64[ns]
 3   status              128975 non-null  category      
 4   fulfillment         128975 non-null  category      
 5   sales_channel       128975 non-null  category      
 6   ship_service_level  128975 non-null  category      
 7   style               128975 non-null  string        
 8   sku                 128975 non-null  string        
 9   category            128975 non-null  category      
 10  size                128975 non-null  category      
 11  asin                128975 non-null  string        
 12  courier_status      122103 non-null  category      
 13  quantity            128975 no

In [ ]:
# Normalize values for categorical and string columns.
uppercase_strings = ['asin', 'style', 'size', 'sku', 'currency', 'ship_country']
lowercase_strings = [col for col in (categorical_columns + string_columns) 
                                 if col not in uppercase_strings]

amazon_sales[uppercase_strings] = (
    amazon_sales[uppercase_strings]
    .apply(lambda x: x.str.strip().str.upper())
)

amazon_sales[lowercase_strings] = (
    amazon_sales[lowercase_strings]
    .apply(lambda x: x.str.strip().str.lower())
)

display(amazon_sales[uppercase_strings].head())

display(amazon_sales[lowercase_strings].head())

,asin,style,size,sku,currency,ship_country
0,B09KXVBD7Z,SET389,S,SET389-KR-NP-S,INR,IN
1,B09K3WFS32,JNE3781,3XL,JNE3781-KR-XXXL,INR,IN
2,B07WV4JV4D,JNE3371,XL,JNE3371-KR-XL,INR,IN
3,B099NRCT7B,J0341,L,J0341-DR-L,INR,IN
4,B098714BZP,JNE3671,3XL,JNE3671-TU-XXXL,INR,IN


,status,fulfillment,sales_channel,ship_service_level,category,courier_status,ship_state,fulfilled_by,order_id,ship_city,promotion_ids,ship_postal_code
0,cancelled,merchant,amazon.in,standard,set,NaN,maharashtra,easy ship,405-8078784-5731545,mumbai,<NA>,400081.0
1,shipped - delivered to buyer,merchant,amazon.in,standard,kurta,shipped,karnataka,easy ship,171-9198151-1101146,bengaluru,amazon plcc free-financing universal merchant ...,560085.0
2,shipped,amazon,amazon.in,expedited,kurta,shipped,maharashtra,NaN,404-0687676-7273146,navi mumbai,in core free shipping 2015/04/08 23-48-5-108,410210.0
3,cancelled,merchant,amazon.in,standard,western dress,NaN,puducherry,easy ship,403-9615377-8133951,puducherry,<NA>,605008.0
4,shipped,amazon,amazon.in,expedited,top,shipped,tamil nadu,NaN,407-1069790-7240320,chennai,<NA>,600073.0


In [ ]:
# Check for duplicates and missing values
print(f"Number of duplicate rows: {amazon_sales.duplicated().sum()} \n")
print(f"Number of duplicate indicies: {amazon_sales.shape[0] - amazon_sales['index'].nunique()} \n")
print(f"Number of duplicate orders ids: {amazon_sales['order_id'].duplicated().sum()} \n")
print(f"Number of null entries by column:\n{amazon_sales.isna().sum()}")

Number of duplicate rows: 0 

Number of duplicate indicies: 0 

Number of duplicate orders ids: 8597 

Number of null entries by column:
index                     0
order_id                  0
date                      0
status                    0
fulfillment               0
sales_channel             0
ship_service_level        0
style                     0
sku                       0
category                  0
size                      0
asin                      0
courier_status         6872
quantity                  0
currency               7795
amount                 7795
ship_city                33
ship_state               33
ship_postal_code         33
ship_country             33
promotion_ids         49153
b2b                       0
fulfilled_by          89698
dtype: int64


We will address null values later in the SQL section. Let's start with duplicates. Looks like they are multiple parts of same order.

In [ ]:
# Inspect duplicate order ids
amazon_sales[amazon_sales['order_id'].duplicated(keep=False)].sort_values('order_id')


,index,order_id,date,status,fulfillment,sales_channel,ship_service_level,style,sku,category,...,quantity,currency,amount,ship_city,ship_state,ship_postal_code,ship_country,promotion_ids,b2b,fulfilled_by
78939,78939,171-0012693-2893106,2022-05-07,cancelled,merchant,amazon.in,standard,NW015,NW015-TP-PJ-S,set,...,0,INR,477.14,sri ganganagar,rajasthan,335001.0,IN,<NA>,False,easy ship
78938,78938,171-0012693-2893106,2022-05-07,cancelled,merchant,amazon.in,standard,NW030,NW030-TP-PJ-S,set,...,0,INR,538.10,sri ganganagar,rajasthan,335001.0,IN,<NA>,False,easy ship
91476,91476,171-0014231-7760344,2022-06-29,pending,amazon,amazon.in,expedited,J0224,J0224-SKD-XS,set,...,1,INR,1364.00,ahmedabad,gujarat,380015.0,IN,<NA>,False,NaN
91475,91475,171-0014231-7760344,2022-06-29,pending,amazon,amazon.in,expedited,SET342,SET342-KR-NP-N-XS,set,...,1,INR,850.00,ahmedabad,gujarat,380015.0,IN,<NA>,False,NaN
24515,24515,171-0015435-1981904,2022-04-16,shipped,amazon,amazon.in,expedited,JNE3611,JNE3611-KR-L,kurta,...,1,INR,459.00,pimpri chinchwad,maharashtra,412101.0,IN,in core free shipping 2015/04/08 23-48-5-108,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74748,74748,408-9929975-2118741,2022-05-11,shipped - delivered to buyer,merchant,amazon.in,standard,SET408,SET408-KR-NP-XXXL,set,...,1,INR,499.00,port blair,andaman & nicobar,744101.0,IN,amazon plcc free-financing universal merchant ...,False,easy ship
36052,36052,408-9951521-7399501,2022-04-09,shipped,amazon,amazon.in,expedited,J0338,J0338-DR-M,western dress,...,1,INR,744.00,secunderabad,telangana,500017.0,IN,in core free shipping 2015/04/08 23-48-5-108,False,NaN
36053,36053,408-9951521-7399501,2022-04-09,shipped,amazon,amazon.in,expedited,J0335,J0335-DR-M,western dress,...,1,INR,807.00,secunderabad,telangana,500017.0,IN,in core free shipping 2015/04/08 23-48-5-108,False,NaN
61445,61445,408-9954515-8573127,2022-05-22,shipped - delivered to buyer,merchant,amazon.in,standard,SET356,SET356-KR-NP-XXL,set,...,1,INR,999.00,shimla,himachal pradesh,171005.0,IN,amazon plcc free-financing universal merchant ...,False,easy ship


To screen for true duplicates, we'll look for orders with same id, item asin, and date

In [ ]:
# Inspect true duplicates
amazon_sales[amazon_sales.duplicated(subset=['order_id', 'asin', 'date'], keep=False)]


,index,order_id,date,status,fulfillment,sales_channel,ship_service_level,style,sku,category,...,quantity,currency,amount,ship_city,ship_state,ship_postal_code,ship_country,promotion_ids,b2b,fulfilled_by
30660,30660,406-0372545-6086735,2022-04-12,cancelled,amazon,amazon.in,expedited,SET197,SET197-KR-NP-L,set,...,0,NaN,NaN,siliguri,west bengal,734008.0,IN,<NA>,False,NaN
30661,30661,406-0372545-6086735,2022-04-12,cancelled,amazon,amazon.in,expedited,SET197,SET197-KR-NP-L,set,...,0,NaN,NaN,siliguri,west bengal,734008.0,IN,<NA>,False,NaN
41291,41291,408-0373839-4433120,2022-04-05,cancelled,amazon,amazon.in,expedited,JNE3501,JNE3501-KR-M,kurta,...,0,NaN,NaN,bengaluru,karnataka,560072.0,IN,<NA>,False,NaN
41292,41292,408-0373839-4433120,2022-04-05,cancelled,amazon,amazon.in,expedited,JNE3501,JNE3501-KR-M,kurta,...,0,NaN,NaN,bengaluru,karnataka,560072.0,IN,<NA>,False,NaN
52330,52330,407-8364731-6449117,2022-05-29,shipped,amazon,amazon.in,expedited,JNE3769,JNE3769-KR-L,kurta,...,0,NaN,NaN,theni,tamil nadu,625579.0,IN,in core free shipping 2015/04/08 23-48-5-108,False,NaN
52333,52333,407-8364731-6449117,2022-05-29,shipped,amazon,amazon.in,expedited,JNE3769,JNE3769-KR-L,kurta,...,1,INR,487.0,theni,tamil nadu,625579.0,IN,in core free shipping 2015/04/08 23-48-5-108,False,NaN
79844,79844,171-9628368-5329958,2022-05-07,cancelled,amazon,amazon.in,expedited,J0329,J0329-KR-L,kurta,...,0,NaN,NaN,ernakulam,kerala,682017.0,IN,<NA>,False,NaN
79845,79845,171-9628368-5329958,2022-05-07,cancelled,amazon,amazon.in,expedited,J0329,J0329-KR-L,kurta,...,0,NaN,NaN,ernakulam,kerala,682017.0,IN,<NA>,False,NaN
85790,85790,171-3249942-2207542,2022-05-03,shipped,amazon,amazon.in,expedited,SET323,SET323-KR-NP-XL,set,...,1,INR,939.0,pune,maharashtra,411057.0,IN,in core free shipping 2015/04/08 23-48-5-108,False,NaN
85791,85791,171-3249942-2207542,2022-05-03,shipped,amazon,amazon.in,expedited,SET323,SET323-KR-NP-XL,set,...,1,INR,939.0,pune,maharashtra,411057.0,IN,in core free shipping 2015/04/08 23-48-5-108,False,NaN


Not bad, only 7 true duplicates. Let's drop keeping last since one entry is missing amount info in first duplicate

In [ ]:
# Drop duplicates
amazon_sales = amazon_sales.drop_duplicates(subset=['order_id', 'asin', 'date'], keep='last')
print(amazon_sales.shape[0])

128968


## Categorical Columns

In [ ]:
# Inspect unique and null values per col
cat_stats = pd.DataFrame({
    'Unique Values': [amazon_sales[col].nunique() for col in categorical_columns],
    'Null Values': [amazon_sales[col].isna().sum() for col in categorical_columns],
}, index=categorical_columns)

display(cat_stats)

,Unique Values,Null Values
status,13,0
fulfillment,2,0
sales_channel,2,0
ship_service_level,2,0
category,9,0
size,11,0
courier_status,3,6872
currency,1,7791
ship_state,47,33
ship_country,1,33


- Currency null values most likely can be filled with INR, but I'll double check against amount and item price info in SQL.
- Having both status and courier status might be redundant. 
- Fulfilled by is ...?

Let's inspect distributions

In [ ]:
# Print unique value counts for categorical columns
for col in categorical_columns:
    print(amazon_sales[col].value_counts(), end="\n\n")

status
shipped                          77800
shipped - delivered to buyer     28769
cancelled                        18329
shipped - returned to seller      1953
shipped - picked up                973
pending                            658
pending - waiting for pick up      281
shipped - returning to seller      145
shipped - out for delivery          35
shipped - rejected by buyer         11
shipping                             8
shipped - lost in transit            5
shipped - damaged                    1
Name: count, dtype: int64

fulfillment
amazon      89691
merchant    39277
Name: count, dtype: int64

sales_channel
amazon.in     128844
non-amazon       124
Name: count, dtype: int64

ship_service_level
expedited    88608
standard     40360
Name: count, dtype: int64

category
set              50281
kurta            49873
western dress    15500
top              10622
ethnic dress      1159
blouse             926
bottom             440
saree              164
dupatta              3
N

- 128942/128975 have india as shipping country. We can fill in for the missing 33. 
- fulfilled_by and fulfillment both mark orders handled by easy ship. There are 39277 of them. We'll reconcile info and drop one of them

In [ ]:
# Fill missing shipping country values. Reconcile fulfillment columns and drop one of them.
amazon_sales['ship_country'] = amazon_sales['ship_country'].fillna("IN")
amazon_sales['fulfillment'] = amazon_sales['fulfillment'].replace({'merchant': 'easy ship'})
amazon_sales.drop(columns='fulfilled_by', inplace=True)

print(amazon_sales['fulfillment'].value_counts())
print(amazon_sales['ship_country'].isna().sum())

fulfillment
amazon       89691
easy ship    39277
Name: count, dtype: int64
0


Next up, we have only 3 dupatta and 164 saree items in the dataset. These categories are not standardized, we'll reassign as ethnic dress for consistency.

In [ ]:
# Relabel categories for better consistency
amazon_sales.loc[
    (amazon_sales['category']=='dupatta') |
    (amazon_sales['category']=='saree')
    , 'category'] ='ethnic dress'

amazon_sales['category'].value_counts()

category
set              50281
kurta            49873
western dress    15500
top              10622
ethnic dress      1326
blouse             926
bottom             440
Name: count, dtype: int64

We're left with status/courier_status and ship state. We'll again try to reconcile the two columns.

In [ ]:
# Inspect status vs courier status.
print(amazon_sales.groupby('status')['courier_status'].value_counts())

status                         courier_status
cancelled                      cancelled          5837
                               unshipped          5631
pending                        unshipped           646
                               shipped              10
                               cancelled             2
pending - waiting for pick up  unshipped           281
shipped                        shipped           77593
                               unshipped           115
                               cancelled            92
shipped - damaged              shipped               1
shipped - delivered to buyer   shipped           28761
shipped - lost in transit      shipped               5
shipped - out for delivery     shipped              35
shipped - picked up            shipped             973
shipped - rejected by buyer    shipped              11
shipped - returned to seller   shipped            1950
shipped - returning to seller  shipped             145
shipping           

Since transportation follows order processing, courier status is more up to date than status. Also looks like orders with cancelled
status have 50/50 unshipped/cancelled courier status, meaning the terms were used interchangebly. So we make the simplifying assumption:

- unshipped --> cancelled

In [ ]:
# Simplify status values and reconcile with courier status
amazon_sales.loc[
    (amazon_sales['courier_status'] == 'unshipped') |
    (amazon_sales['courier_status'] == 'cancelled'),
'status'] = 'cancelled'

amazon_sales.loc[
    (amazon_sales['courier_status'] == 'shipped') &
    (amazon_sales['status'] == 'pending'),
'status'] = 'shipped'

print(amazon_sales.groupby('status')['courier_status'].value_counts())
print(f"\n{amazon_sales['status'].value_counts()}")

status                         courier_status
cancelled                      unshipped          6681
                               cancelled          5931
shipped                        shipped           77603
shipped - damaged              shipped               1
shipped - delivered to buyer   shipped           28761
shipped - lost in transit      shipped               5
shipped - out for delivery     shipped              35
shipped - picked up            shipped             973
shipped - rejected by buyer    shipped              11
shipped - returned to seller   shipped            1950
shipped - returning to seller  shipped             145
Name: count, dtype: int64

status
shipped                          77603
shipped - delivered to buyer     28769
cancelled                        19473
shipped - returned to seller      1953
shipped - picked up                973
shipped - returning to seller      145
shipped - out for delivery          35
shipped - rejected by buyer         11
shi

Considering ambiguity in cancelled shipment status, we won't be able to meaningfully distinguish between orders cancelled before shipment and orders we had to ship back for cost analysis. Instead, let's focus on analyzing consumer choices vis a vie cancellation. For this reason, I'll combine different categories as cancelled

In [ ]:
# Combine status as cancelled or returned. Drop statuses with 0 value count
combine_status = {
    "shipped - returned to seller": "cancelled",
    "shipped - returning to seller": "cancelled",
    "shipped - rejected by buyer": "cancelled",
    "shipped - lost in transit": "cancelled",
    "shipped - damaged": "cancelled"
}
amazon_sales['status'] = amazon_sales['status'].replace(combine_status)
amazon_sales['status'] = amazon_sales['status'].replace({'cancelled': 'cancelled or returned'})

# All of these orders had updates in courier_status and were not updated in status. We are left with 0 order count for each. 
status_to_drop = ['pending', 'pending - waiting for pick up', 'shipping']
amazon_sales = amazon_sales[~amazon_sales['status'].isin(status_to_drop)]

print(amazon_sales['status'].value_counts())
print(amazon_sales.shape[0])

status
shipped                         77603
shipped - delivered to buyer    28769
cancelled or returned           21588
shipped - picked up               973
shipped - out for delivery         35
Name: count, dtype: int64
128968


Looks good. Let's drop courier status

In [ ]:
# Drop courier_status
amazon_sales.drop(columns='courier_status', inplace=True)
display(amazon_sales.head())

,index,order_id,date,status,fulfillment,sales_channel,ship_service_level,style,sku,category,...,asin,quantity,currency,amount,ship_city,ship_state,ship_postal_code,ship_country,promotion_ids,b2b
0,0,405-8078784-5731545,2022-04-30,cancelled or returned,easy ship,amazon.in,standard,SET389,SET389-KR-NP-S,set,...,B09KXVBD7Z,0,INR,647.62,mumbai,maharashtra,400081.0,IN,<NA>,False
1,1,171-9198151-1101146,2022-04-30,shipped - delivered to buyer,easy ship,amazon.in,standard,JNE3781,JNE3781-KR-XXXL,kurta,...,B09K3WFS32,1,INR,406.00,bengaluru,karnataka,560085.0,IN,amazon plcc free-financing universal merchant ...,False
2,2,404-0687676-7273146,2022-04-30,shipped,amazon,amazon.in,expedited,JNE3371,JNE3371-KR-XL,kurta,...,B07WV4JV4D,1,INR,329.00,navi mumbai,maharashtra,410210.0,IN,in core free shipping 2015/04/08 23-48-5-108,True
3,3,403-9615377-8133951,2022-04-30,cancelled or returned,easy ship,amazon.in,standard,J0341,J0341-DR-L,western dress,...,B099NRCT7B,0,INR,753.33,puducherry,puducherry,605008.0,IN,<NA>,False
4,4,407-1069790-7240320,2022-04-30,shipped,amazon,amazon.in,expedited,JNE3671,JNE3671-TU-XXXL,top,...,B098714BZP,1,INR,574.00,chennai,tamil nadu,600073.0,IN,<NA>,False


Next, ship state. Many of the values are territories, so let's account for both

In [ ]:
# Find incorrectly labeled states/territories
states_territories = '''Andhra Pradesh, Arunachal Pradesh, Assam, Bihar, Chhattisgarh, Goa, Gujarat, Haryana, 
Himachal Pradesh, Jharkhand, Karnataka, Kerala, Madhya Pradesh, Maharashtra, Manipur, Meghalaya, 
Mizoram, Nagaland, Odisha, Punjab, Rajasthan, Sikkim, Tamil Nadu, Telangana, Tripura, Uttar Pradesh, 
Uttarakhand, West Bengal, Andaman and Nicobar Islands, Chandigarh, Dadra and Nagar Haveli and Daman and Diu, 
Delhi, Jammu and Kashmir, Ladakh, Lakshadweep, Puducherry'''
states_territories = sorted(states_territories.replace('\n', '').lower().split(', '))
print(states_territories)
print(len(states_territories))
amazon_sales = amazon_sales.rename(columns={'ship_state': 'ship_state_or_territory'})

incorrect_states_labels = (amazon_sales[
    ~amazon_sales['ship_state_or_territory']
    .isin(states_territories)]['ship_state_or_territory']
.unique())
print(f"\n Incorrectly labeled states/territories:\n {incorrect_states_labels}")

['andaman and nicobar islands', 'andhra pradesh', 'arunachal pradesh', 'assam', 'bihar', 'chandigarh', 'chhattisgarh', 'dadra and nagar haveli and daman and diu', 'delhi', 'goa', 'gujarat', 'haryana', 'himachal pradesh', 'jammu and kashmir', 'jharkhand', 'karnataka', 'kerala', 'ladakh', 'lakshadweep', 'madhya pradesh', 'maharashtra', 'manipur', 'meghalaya', 'mizoram', 'nagaland', 'odisha', 'puducherry', 'punjab', 'rajasthan', 'sikkim', 'tamil nadu', 'telangana', 'tripura', 'uttar pradesh', 'uttarakhand', 'west bengal']
36

 Incorrectly labeled states/territories:
 ['jammu & kashmir' 'dadra and nagar' nan 'andaman & nicobar' 'rajshthan'
 'nl' 'new delhi' 'punjab/mohali/zirakpur' 'rj' 'orissa' 'pb' 'apo' 'ar'
 'pondicherry' 'rajsthan']


In [37]:
# Update values using a predictive model
update = {
    "jammu & kashmir": "jammu and kashmir",
    "dadra and nagar": "dadra and nagar haveli and daman and diu",
    "andaman & nicobar": "andaman and nicobar islands",
    "rajshthan": "rajasthan",
    "nl": "nagaland",
    "new delhi": "delhi",
    "punjab/mohali/zirakpur": "punjab",
    "rj": "rajasthan",
    "orissa": "odisha",
    "pb": "punjab",
    "apo": "unknown",
    "ar": "arunachal pradesh",
    "pondicherry": "puducherry",
    "rajsthan": "rajasthan"
}
amazon_sales['ship_state_or_territory'] = amazon_sales['ship_state_or_territory'].replace(update)

# Fill na values and the one address with "apo" as unknown.
amazon_sales[['ship_state_or_territory', 'ship_city', 'ship_postal_code']] = (
    amazon_sales[['ship_state_or_territory', 'ship_city', 'ship_postal_code']]
    .fillna('unknown')
)

print(amazon_sales['ship_state_or_territory'].isna().sum())
print(amazon_sales['ship_state_or_territory'].nunique())
print(sorted(amazon_sales['ship_state_or_territory'].unique()))

0
37
['andaman and nicobar islands', 'andhra pradesh', 'arunachal pradesh', 'assam', 'bihar', 'chandigarh', 'chhattisgarh', 'dadra and nagar haveli and daman and diu', 'delhi', 'goa', 'gujarat', 'haryana', 'himachal pradesh', 'jammu and kashmir', 'jharkhand', 'karnataka', 'kerala', 'ladakh', 'lakshadweep', 'madhya pradesh', 'maharashtra', 'manipur', 'meghalaya', 'mizoram', 'nagaland', 'odisha', 'puducherry', 'punjab', 'rajasthan', 'sikkim', 'tamil nadu', 'telangana', 'tripura', 'unknown', 'uttar pradesh', 'uttarakhand', 'west bengal']


## String Columns

In [ ]:
# Check number of unique values for each column
for col in string_columns:
    print(f"{col:<20} {amazon_sales[col].nunique()} unique values")

order_id             120378 unique values
style                1377 unique values
sku                  7195 unique values
asin                 7190 unique values
ship_city            7298 unique values
promotion_ids        5787 unique values
ship_postal_code     9460 unique values


Next, we will:

- Check style, sku, and asin column values for unexpected characters. 
- Check tail end of value_counts sample for bogus values.

In [ ]:
# Check for unusual values in string columns
for col in ['style', 'asin', 'sku']:
    print(amazon_sales[amazon_sales[col].str.contains(r'[^a-zA-Z0-9\s-]', na=True)], end="\n\n")

for col in ['style', 'asin', 'sku']:
    print(amazon_sales[col].value_counts().tail(10))

Empty DataFrame
Columns: [index, order_id, date, status, fulfillment, sales_channel, ship_service_level, style, sku, category, size, asin, quantity, currency, amount, ship_city, ship_state_or_territory, ship_postal_code, ship_country, promotion_ids, b2b]
Index: []

[0 rows x 21 columns]

Empty DataFrame
Columns: [index, order_id, date, status, fulfillment, sales_channel, ship_service_level, style, sku, category, size, asin, quantity, currency, amount, ship_city, ship_state_or_territory, ship_postal_code, ship_country, promotion_ids, b2b]
Index: []

[0 rows x 21 columns]

Empty DataFrame
Columns: [index, order_id, date, status, fulfillment, sales_channel, ship_service_level, style, sku, category, size, asin, quantity, currency, amount, ship_city, ship_state_or_territory, ship_postal_code, ship_country, promotion_ids, b2b]
Index: []

[0 rows x 21 columns]

style
CH207      1
J0115      1
SAR140     1
DPT052     1
DPT041     1
SAR182     1
DPT032     1
SET446     1
JNE3450    1
SET045    

Looks good. Next up, postal codes still have a decimal digit from being float type. Let's fix that and check ship_city and postal code for unusual characters

In [ ]:
# Fix postal code decimal place and check for unusual values
amazon_sales['ship_postal_code'] = amazon_sales['ship_postal_code'].str.replace('\.0$', '', regex=True)
display(amazon_sales[amazon_sales['ship_postal_code'].str.contains(r'[^0-9\s]', regex=True)])
display(amazon_sales[amazon_sales['ship_city'].str.contains(r'[^a-zA-Z\s]', regex=True)])

,index,order_id,date,status,fulfillment,sales_channel,ship_service_level,style,sku,category,...,asin,quantity,currency,amount,ship_city,ship_state_or_territory,ship_postal_code,ship_country,promotion_ids,b2b
1871,1871,404-0566904-4825137,2022-04-29,shipped,amazon,amazon.in,expedited,J0301,J0301-TP-L,top,...,B099S6795L,1,INR,493.0,unknown,unknown,unknown,IN,in core free shipping 2015/04/08 23-48-5-108,False
1872,1872,404-0566904-4825137,2022-04-29,shipped,amazon,amazon.in,expedited,JNE3697,JNE3697-KR-L,kurta,...,B09811Y7WM,1,INR,458.0,unknown,unknown,unknown,IN,in core free shipping 2015/04/08 23-48-5-108,False
8752,8752,406-4003386-8768363,2022-04-25,shipped,amazon,amazon.in,expedited,JNE3376,JNE3376-KR-M,kurta,...,B082W7HW86,1,INR,432.0,unknown,unknown,unknown,IN,<NA>,False
11215,11215,402-0107720-7057168,2022-04-23,shipped,amazon,amazon.in,expedited,J0003,J0003-SET-S,set,...,B0894X27FC,1,INR,654.0,unknown,unknown,unknown,IN,in core free shipping 2015/04/08 23-48-5-108,False
13252,13252,407-4532637-8415521,2022-04-22,cancelled or returned,easy ship,amazon.in,standard,JNE3405,JNE3405-KR-S,kurta,...,B081WX4G4Q,0,INR,380.0,unknown,unknown,unknown,IN,<NA>,False
15688,15688,404-9229894-8608305,2022-04-21,shipped,amazon,amazon.in,expedited,JNE3510,JNE3510-KR-M,kurta,...,B08WPR5MCB,1,INR,442.0,unknown,unknown,unknown,IN,in core free shipping 2015/04/08 23-48-5-108,False
16787,16787,402-4919636-4333150,2022-04-20,shipped - delivered to buyer,easy ship,amazon.in,standard,JNE1233,JNE1233-BLUE-KR-031-XXXL,kurta,...,B07GPGP46N,1,INR,376.0,unknown,unknown,unknown,IN,amazon plcc free-financing universal merchant ...,False
18351,18351,405-4927647-8064368,2022-04-19,shipped,amazon,amazon.in,expedited,J0230,J0230-SKD-XS,set,...,B08XNHXSFZ,1,INR,1112.0,unknown,unknown,unknown,IN,in core free shipping 2015/04/08 23-48-5-108,False
22930,22930,402-8628677-0457954,2022-04-16,cancelled or returned,easy ship,amazon.in,standard,J0003,J0003-SET-S,set,...,B0894X27FC,1,INR,654.0,unknown,unknown,unknown,IN,amazon plcc free-financing universal merchant ...,False
24986,24986,406-4079063-8291520,2022-04-15,cancelled or returned,amazon,amazon.in,expedited,JNE3567,JNE3567-KR-XXL,kurta,...,B08KRYCC8J,1,INR,399.0,unknown,unknown,unknown,IN,<NA>,False


,index,order_id,date,status,fulfillment,sales_channel,ship_service_level,style,sku,category,...,asin,quantity,currency,amount,ship_city,ship_state_or_territory,ship_postal_code,ship_country,promotion_ids,b2b
12,12,405-5513694-8146768,2022-04-30,shipped - delivered to buyer,easy ship,amazon.in,standard,JNE3405,JNE3405-KR-XS,kurta,...,B081XCMYXJ,1,INR,399.0,amravati.,maharashtra,444606,IN,amazon plcc free-financing universal merchant ...,False
122,122,406-8961316-8137943,2022-04-30,shipped - delivered to buyer,easy ship,amazon.in,standard,SET324,SET324-KR-NP-S,set,...,B09NQ4CZ65,1,INR,597.0,"guwahati, kamrup (m)",assam,781014,IN,amazon plcc free-financing universal merchant ...,False
123,123,406-8961316-8137943,2022-04-30,shipped - delivered to buyer,easy ship,amazon.in,standard,SET332,SET332-KR-PP-S,set,...,B09NQ33VW2,1,INR,549.0,"guwahati, kamrup (m)",assam,781014,IN,amazon plcc free-financing universal merchant ...,False
155,155,402-8078642-8565124,2022-04-30,cancelled or returned,amazon,amazon.in,expedited,JNE3887,JNE3887-KR-XL,kurta,...,B09TZVXQ1J,0,NaN,NaN,puri-2,odisha,752002,IN,<NA>,False
288,288,405-3441326-1516368,2022-04-30,shipped,amazon,amazon.in,expedited,J0118,J0118-TP-XXXL,top,...,B08N43MK22,1,INR,518.0,kolkata 700034,west bengal,700034,IN,in core free shipping 2015/04/08 23-48-5-108,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128770,128770,403-0380725-0352329,2022-06-01,shipped,amazon,amazon.in,expedited,JNE3703,JNE3703-KR-XL,kurta,...,B099NNYQ1K,1,INR,292.0,new delhi 110032,delhi,110032,IN,<NA>,False
128855,128855,402-1958677-2521967,2022-06-01,cancelled or returned,easy ship,amazon.in,standard,JNE3568,JNE3568-KR-M,kurta,...,B08KRZRSCL,0,NaN,NaN,cherukunnu (p.o),kerala,670301,IN,<NA>,False
128946,128946,404-4165392-8631530,2022-05-31,shipped,amazon,amazon.in,expedited,JNE3560,JNE3560-KR-M,kurta,...,B08PCVMF6R,1,INR,544.0,"gulabpura, distt bhilwara",rajasthan,311021,IN,in core free shipping 2015/04/08 23-48-5-108,False
128965,128965,408-5154281-4593912,2022-05-31,cancelled or returned,amazon,amazon.in,expedited,J0119,J0119-TP-XXXL,top,...,B08RYPRVPV,1,INR,574.0,prayagraj (allahabad),uttar pradesh,211007,IN,<NA>,False


Postal code looks good. Ship city needs advanced querying to fix and we will address it in sql. Let's check promotion_ids

In [ ]:
# Inspect promotion_ids
pd.set_option('display.max_colwidth', None)
display(amazon_sales[
    amazon_sales['promotion_ids']
    .str.contains('amazon', case=False, na=False)]
    ['promotion_ids'].head(1))

1    amazon plcc free-financing universal merchant aat-wnktbo3k27ejc,amazon plcc free-financing universal merchant aat-qx3uccjeskpa2,amazon plcc free-financing universal merchant aat-5qq7biyyqedn2,amazon plcc free-financing universal merchant aat-dsj2qrxxwxvmq,amazon plcc free-financing universal merchant aat-cxjhmc2yjuk76,amazon plcc free-financing universal merchant aat-cc4favtyr4x7c,amazon plcc free-financing universal merchant aat-xxrcw6nzepzi4,amazon plcc free-financing universal merchant aat-cxnslnbrofdw4,amazon plcc free-financing universal merchant aat-r7gxnzwistrfa,amazon plcc free-financing universal merchant aat-wsjldn3x7kemo,amazon plcc free-financing universal merchant aat-vl6fgqvgqvxus,amazon plcc free-financing universal merchant aat-eokpwfwyw7y6i,amazon plcc free-financing universal merchant aat-zyl5upunw6t62,amazon plcc free-financing universal merchant aat-xvpicchrwdcai,amazon plcc free-financing universal merchant aat-etxq3xxwmrxbg,amazon plcc free-financing universa

,index,order_id,date,status,fulfillment,sales_channel,ship_service_level,style,sku,category,...,asin,quantity,currency,amount,ship_city,ship_state_or_territory,ship_postal_code,ship_country,promotion_ids,b2b


It looks like this column is filled incorrectly. For example, this order has a single item, but it has more than 10
promotional codes applied in a repetitive fashion. Since we can't reliably differentiate promotional codes,
we will focus our analysis on whether there is a promotion. i.e. we will convert to boolean

In [ ]:
# Before we convert. Any promotion ids with normal string length?
display(amazon_sales[amazon_sales['promotion_ids'].str.len() < 10])

# Convert to boolean
amazon_sales['promotion_ids'] = amazon_sales['promotion_ids'].notna()

## Numerical Columns

In [ ]:
# Check number of null entries and entries with 0 value for amount and quantity
numeric_columns = ["quantity", "amount"]
print(f"Count of null entries:\n{amazon_sales[numeric_columns].isna().sum()}\n")
print(f"Quantity = 0: {(amazon_sales['quantity'] == 0).sum():>8}")
print(f"Amount = 0: {(amazon_sales['amount'] == 0).sum():>10}\n")


print(amazon_sales[numeric_columns].describe())

Count of null entries:
quantity       0
amount      7791
dtype: int64

Quantity = 0:    12803
Amount = 0:       2343

            quantity         amount
count  128968.000000  121177.000000
mean        0.904457     648.555776
std         0.313321     281.209851
min         0.000000       0.000000
25%         1.000000     449.000000
50%         1.000000     605.000000
75%         1.000000     788.000000
max        15.000000    5584.000000


In [44]:
# How many of the orders missing payment infor are cancelled?

amazon_sales[
    (amazon_sales['quantity'] == 0) |
    (amazon_sales['quantity'].isna()) |
    (amazon_sales['amount'] == 0) |
    (amazon_sales['amount'].isna()) 
]['status'].value_counts()

status
cancelled or returned           13000
shipped                          1518
shipped - delivered to buyer      724
shipped - picked up                28
Name: count, dtype: int64

While the majority of the orders missing payment info have cancelled/returned status, I would like to recover this information to analyze missed revenue opporunity. We will
recover this information in SQL.

In [ ]:
# Also ooks like all orders made through non-amazon sales channel are missing currency and amount.
print(amazon_sales['sales_channel'].value_counts(), "\n")
print(amazon_sales[amazon_sales['sales_channel'] == "non-amazon"].isna().sum())

sales_channel
amazon.in     128844
non-amazon       124
Name: count, dtype: int64 

index                        0
order_id                     0
date                         0
status                       0
fulfillment                  0
sales_channel                0
ship_service_level           0
style                        0
sku                          0
category                     0
size                         0
asin                         0
quantity                     0
currency                   124
amount                     124
ship_city                    0
ship_state_or_territory      0
ship_postal_code             0
ship_country                 0
promotion_ids                0
b2b                          0
dtype: int64


In [46]:
# final check on datatypes and columns we modiefied before exporting to sql
amazon_sales.info()
columns_to_inspect = ['status', 'fulfillment', 'ship_state_or_territory', 'ship_country', 'promotion_ids']
for col in columns_to_inspect:
    print("\n", amazon_sales[col].value_counts())

<class 'pandas.core.frame.DataFrame'>
Index: 128968 entries, 0 to 128974
Data columns (total 21 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   index                    128968 non-null  int64         
 1   order_id                 128968 non-null  string        
 2   date                     128968 non-null  datetime64[ns]
 3   status                   128968 non-null  object        
 4   fulfillment              128968 non-null  object        
 5   sales_channel            128968 non-null  object        
 6   ship_service_level       128968 non-null  object        
 7   style                    128968 non-null  string        
 8   sku                      128968 non-null  string        
 9   category                 128968 non-null  object        
 10  size                     128968 non-null  object        
 11  asin                     128968 non-null  string        
 12  quantity             

In [47]:
# Write to csv file
timestamp = pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')
amazon_sales.to_csv(f'data/amazon_sales_cleaned_{timestamp}.csv', index=False)